In [ ]:
import logging
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from markdownify import markdownify as md
import time
from openai import OpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from typing import List, Optional

load_dotenv()

def setup_selenium():
    """Sets up Selenium with headless Chrome."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def fetch_html(driver, url, timeout=30):
    """Fetches HTML content from URL using Selenium."""
    try:
        driver.get(url)
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CLASS_NAME, "note"))
        )
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        return driver.page_source
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

def extract_reviews_to_markdown(url):
    """Extracts the paper abstract and all reviews from a given OpenReview URL and returns them as markdown text."""
    driver = setup_selenium()
    try:
        html = fetch_html(driver, url)
        if not html:
            return "Error: Could not fetch page content"
        
        soup = BeautifulSoup(html, "html.parser")
        
        # Get paper title
        title = soup.find("h2", class_="citation_title")
        markdown_text = f"# {title.text.strip() if title else 'Unknown Paper'}\n\n"
        
        # Extract paper abstract
        abstract = ""
        abstract_header = soup.find("h4", string=re.compile(r"Abstract", re.I))
        if abstract_header:
            abstract_para = abstract_header.find_next_sibling("p")
            if abstract_para:
                abstract = abstract_para.text.strip()
        if abstract:
            markdown_text += f"## Abstract\n\n{abstract}\n\n"
        
        # Find all review notes
        reviews = []
        for note in soup.find_all("div", class_="note"):
            invitation = note.find("span", class_="invitation")
            if not invitation:
                continue
                
            invitation_text = invitation.text.strip()
            if "Official Review" in invitation_text:
                # Get reviewer
                signature = note.find("span", class_="signatures")
                reviewer = signature.text.strip() if signature else "Anonymous"
                
                # Get review content
                content = note.find("div", class_="note-content")
                if content:
                    review_text = md(str(content))
                    reviews.append((reviewer, review_text))
        
        # Format reviews as markdown
        for i, (reviewer, review) in enumerate(reviews, 1):
            markdown_text += f"## Review {i}\n\n"
            markdown_text += f"**Reviewer:** {reviewer}\n\n"
            markdown_text += f"{review}\n\n"
            markdown_text += "---\n\n"
        
        return markdown_text
    
    finally:
        driver.quit()
        
class ReviewQualityAnalyzer(BaseModel):
    language_quality_and_professionalism_paragraph: str
    language_quality_and_professionalism_score: int
    internal_consistency_paragraph: str
    internal_consistency_score: int
    constructiveness_of_criticism_paragraph: str
    constructiveness_of_criticism_score: int
    
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Example usage
url = "https://openreview.net/forum?id=Iyve2ycvGZ"  # Replace with your URL
markdown_output = extract_reviews_to_markdown(url)
print(markdown_output)

response = client.beta.chat.completions.parse(
    model="gemini-2.0-flash",
    messages=[
        {
            "role": "system", 
            "content": (
                "You are an expert review analysis assistant. You will be provided with a paper review "
                "in Markdown format. Your task is to evaluate this review comprehensively under three criteria: "
                "1) Language quality and professionalism, 2) Internal consistency, and 3) Constructiveness of criticism. "
                "For each criterion, assign a score between "
                "1 (lowest) and 100 (highest). Ensure your response corresponds to the structure of the ReviewQualityAnalyzer model."
            )
        },
        {
            "role": "user", 
            "content": f"{markdown_output}"
        }
    ],
    response_format=ReviewQualityAnalyzer
)

a = response.choices[0].message.content

print(a)

# Bellman Optimal Stepsize Straightening of Flow-Matching Models

## Review 1

**Reviewer:** by Reviewer TqqW

**Summary:** The author introduces the Bellman Optimal Step\-size Straightening (BOSS) technique, a method for distilling flow\-matching generative models that enhances the efficiency of image sampling within computational budget constraints. BOSS utilizes dynamic programming to optimize step sizes in a pretrained network and refines the velocity network to straighten generation paths. In this paper, the proposed method has been extensively evaluated on image generation tasks, showing that it significantly improves resource efficiency and maintains high image quality. This approach serves to reconcile the intensive computational demands of flow\-matching models with low\-resource availability, contributing to the sustainable development of artificial intelligence by reducing computational expenses and environmental impacts.


**Soundness:** 3 good**Presentation:** 3 good**Cont

BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': 'API key not valid. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API key not valid. Please pass a valid API key.'}]}}]

In [48]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="o3-mini-2025-1-31",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

NotFoundError: Error code: 404 - {'error': {'message': 'The model `o3-mini-2025-1-31` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [47]:
print(event)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']
